# All About Qubits

In [1]:
import numpy as np
import pennylane as qml

**What is Quantum Computing?**

We already know how computers use "bits" to store and process information? Like, a bit can be either 0 or 1. Well, quantum computing
is different. It uses special bits called "qubits". These qubits behave differently compared to classical bits - they can exist
in many states at the same time.

**What's Superposition?**

Let's imagine we have a coin. Heads or tails, it's one or the other, but with a qubit it's like the coin is both heads AND tails at the same
time. This property of being able to be in multiple states simultaneously is called superposition.

**How Do We Describe Qubits Mathematically?**

We represent qubits as vectors in a complex vector space. Think of it like this: if we have two states, $|0\rangle$ and $|1\rangle$, 
the qubit can be a combination of both at the same time.

**The `normalize_state` function**

This function takes two complex numbers: `alpha` and `beta`, representing the amplitudes associated with $|0\rangle$ and $|1\rangle$, respectively.
Its purpose is to return a normalized vector representation of these states.

In [2]:
# Here are the vector representations of |0> and |1>, for convenience
ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])

def normalize_state(alpha, beta):
    # CREATE A VECTOR [a', b'] BASED ON alpha AND beta SUCH THAT |a'|^2 + |b'|^2 = 1
    res = np.array([alpha*np.sqrt(1/(abs(alpha)**2 + abs(beta)**2)), beta*np.sqrt(1/(abs(alpha)**2 + abs(beta)**2))])
    # RETURN A VECTOR
    return res

**What's the Inner Product?**

The inner product is a way to measure how similar or different two vectors are. Imagine taking a dot product between
two vectors that represent the qubits. This gives us an idea of how much they have in common or differ from each other.

**The `inner_product` function**

We define a function `inner_product` that takes two qubit states (represented as NumPy arrays) and returns their
inner product.

The function uses NumPy's `inner` function to compute the dot product between the conjugate of `state_1` and `state_2`.


In [3]:
def inner_product(state_1, state_2):
    # COMPUTE AND RETURN THE INNER PRODUCT
    inner = np.inner(np.conjugate(state_1), state_2)
    return inner

# Test your results with this code
ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])

print(f"<0|0> = {inner_product(ket_0, ket_0)}")
print(f"<0|1> = {inner_product(ket_0, ket_1)}")
print(f"<1|0> = {inner_product(ket_1, ket_0)}")
print(f"<1|1> = {inner_product(ket_1, ket_1)}")

<0|0> = 1
<0|1> = 0
<1|0> = 0
<1|1> = 1


**The `measure_state` function**

We're simulating a quantum measurement process. In this context, "measurement" means determining the state of a qubit (quantum bit) by
observing it.

It takes two inputs:

1. `state`: A normalized qubit state vector represented as a NumPy array `[a, b]`, where `a` and `b` are complex numbers.
2. `num_meas`: An integer indicating the number of measurements to take.

This function simulates a measurement process by generating a set of measurement outcomes according to the probability distribution defined
by the input state.

In [4]:
def measure_state(state, num_meas):
    # COMPUTE THE MEASUREMENT OUTCOME PROBABILITIES
    probs = [abs(state[0])**2, abs(state[1])**2]
    # RETURN A LIST OF SAMPLE MEASUREMENT OUTCOMES
    measurements = np.random.choice(a=[0,1], size = num_meas, p=probs)
    return measurements

In [5]:
measure_state([0.8, 0.6], 20)

array([1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0])

**The `apply_u` function**

We're defining a quantum operation, represented by the matrix `U`. This matrix is used to transform an input state into a new output state.

This function applies the quantum operation `U` to the input state `state`. The resulting output state is then returned.

In [6]:
U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)


def apply_u(state):
    # APPLY U TO THE INPUT STATE AND RETURN THE NEW STATE
    res = U @ state
    return res

apply_u([0.8,0.6])

array([0.98994949, 0.14142136])

In [7]:
U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)


def initialize_state():
    # PREPARE THE STATE |0>
    return np.array([1,0])


def apply_u(state):
    return np.dot(U, state)


def measure_state(state, num_meas):
    p_alpha = np.abs(state[0]) ** 2
    p_beta = np.abs(state[1]) ** 2
    meas_outcome = np.random.choice([0, 1], p=[p_alpha, p_beta], size=num_meas)
    return meas_outcome


def quantum_algorithm():
    # PREPARE THE STATE, APPLY U, THEN TAKE 100 MEASUREMENT SAMPLES
    return measure_state(apply_u(initialize_state()), 100)

# Quantum Circuits

In a quantum circuit, we're working with a series of quantum gates that manipulate qubits. We can think of it like a staff filled
with musical notes, to express its artistic potential. 

**Visualizing Quantum Algorithms**

Imagine a flowchart for quantum algorithms. Each step is represented by a specific gate or operation applied to the qubits. This visual
representation helps us understand how the algorithm works and makes it easier to debug.

**Gates and Operations**

These are the quantum equivalent of logic gates in digital circuits. Here are some common ones:

* **Hadamard gate (H)**: Creates a superposition of 0 and 1.
* **Pauli-X gate**: Flips the state of a qubit.
* **Controlled-NOT gate (CNOT)**: A two-qubit operation that applies an X gate to one qubit based on the state of another.

**Measurements**

These are the points in the circuit where we observe the state of our qubits. Measurements "collapse" superpositions into definite outcomes,
which is how we extract information from quantum systems.

The following code snippets show how to create a `device` and a `QNode` in *Pennylane*, how to use a set of quantum gates and how to interpret the circuit depth.

In [8]:
def my_circuit(theta, phi):
    qml.CNOT(wires=[0, 1])
    qml.RX(theta, wires=2)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[2, 0])
    qml.RY(phi, wires=1)

    # This is the measurement; we return the probabilities of all possible output states
    # You'll learn more about what types of measurements are available in a later node
    return qml.probs(wires=[0, 1, 2])


In [9]:
# This creates a device with three wires on which PennyLane can run computations
dev = qml.device("default.qubit", wires=3)

def my_circuit(theta, phi, omega):
    # IMPLEMENT THE CIRCUIT BY ADDING THE GATES
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])

    return qml.probs(wires=[0, 1, 2])

# This creates a QNode, binding the function and device
my_qnode = qml.QNode(my_circuit, dev)

# We set up some values for the input parameters
theta, phi, omega = 0.1, 0.2, 0.3

# Now we can execute the QNode by calling it like we would a regular function
my_qnode(theta, phi, omega)

tensor([9.87560268e-01, 0.00000000e+00, 0.00000000e+00, 2.47302134e-03,
        2.48960206e-05, 0.00000000e+00, 0.00000000e+00, 9.94181506e-03], requires_grad=True)

In [10]:
dev = qml.device("default.qubit", wires=3)
# DECORATE THE FUNCTION BELOW TO TURN IT INTO A QNODE

@qml.qnode(dev)
def my_circuit(theta, phi, omega):
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])
    return qml.probs(wires=[0, 1, 2])


theta, phi, omega = 0.1, 0.2, 0.3

# RUN THE QNODE WITH THE PROVIDED PARAMETERS
my_circuit(theta, phi, omega)

tensor([9.87560268e-01, 0.00000000e+00, 0.00000000e+00, 2.47302134e-03,
        2.48960206e-05, 0.00000000e+00, 0.00000000e+00, 9.94181506e-03], requires_grad=True)

In [11]:
dev = qml.device("default.qubit", wires=3)


@qml.qnode(dev)
def my_circuit(theta, phi, omega):
    qml.RX(theta, wires=0)
    qml.RY(phi, wires=1)
    qml.RZ(omega, wires=2)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 0])
    return qml.probs(wires=[0, 1, 2])

# FILL IN THE CORRECT CIRCUIT DEPTH
depth = 4

# Unitary Matrices

In quantum mechanics, a **unitary operator** is an operator that preserves the norm of a vector when applied to it. In other words, if
we have a vector $|\psi\rangle$, applying a unitary operator $U$ to it will result in another vector $|\phi\rangle = U |\psi\rangle$, such that the norm (or length) of $|\phi\rangle$ is equal to the norm of $|\psi\rangle$.

Unitary operators are important in quantum mechanics because they describe transformations that leave the probability amplitudes of
states unchanged. In other words, unitaries are "reversible" and do not destroy any information about the original state.

In matrix form, a unitary operator is represented by a square matrix $U$ such that:

1. The matrix $U$ has complex entries.
2. The transpose (or conjugate transpose) of $U$, denoted as $U^{\dagger}$ or $U^*$, satisfies: $UU^{\dagger} = U^{\dagger}U = I$

In [12]:
dev = qml.device("default.qubit", wires=1)

U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

@qml.qnode(dev)
def apply_u():
    # USE QubitUnitary TO APPLY U TO THE QUBIT
    qml.QubitUnitary(U, wires=0)
    # Return the state
    return qml.state()

A **parametrized unitary** is a unitary operator that depends on one or more parameters (angles).

In [13]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def apply_u_as_rot(phi, theta, omega):
    # APPLY A ROT GATE USING THE PROVIDED INPUT PARAMETERS
    qml.Rot(phi, theta, omega, wires=0)
    # RETURN THE QUANTUM STATE VECTOR

    return qml.state()


# X and H

**X gate (Pauli-X)**

The X gate is a quantum gate that flips the state of a qubit. In other words, it takes the $|0\rangle$ state to the $|1\rangle$ state and vice versa.

Mathematically, the X gate can be represented by the following matrix:

$X = \begin{bmatrix} 0 & 1 \\  1 & 0 \end{bmatrix}$

**Hadamard gate (H)**

The Hadamard gate is a quantum gate that creates a superposition of two states. Specifically, it takes the $|0\rangle$ state to an equal
superposition of $|0\rangle$ and $|1\rangle$:

$|0\rangle = {1 \over \sqrt{2}} \cdot (|0\rangle + |1\rangle)$

Mathematically, the Hadamard gate can be represented by the following matrix:

$H = {1 \over \sqrt{2}} \cdot \begin{bmatrix}1 & 1 \\  1 & -1 \end{bmatrix}$


In [14]:
dev = qml.device("default.qubit", wires=1)

U = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

@qml.qnode(dev)
def varied_initial_state(state):
    # KEEP THE QUBIT IN |0> OR CHANGE IT TO |1> DEPENDING ON THE state PARAMETER
    if state:
        qml.PauliX(wires=0)
    # APPLY U TO THE STATE
    qml.QubitUnitary(U, wires=0)
    
    return qml.state()

In [15]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def apply_hadamard():
    # APPLY THE HADAMARD GATE
    qml.Hadamard(wires=0)
    # RETURN THE STATE
    return qml.state()

In [16]:
# CREATE A DEVICE
dev = qml.device("default.qubit", wires=1)
# CREATE A QNODE CALLED apply_hxh THAT APPLIES THE CIRCUIT ABOVE

@qml.qnode(dev)
def apply_hxh(state):
    if state:
        qml.PauliX(wires=0)

    qml.Hadamard(wires=0)
    qml.PauliX(wires=0)
    qml.Hadamard(wires=0)

    return qml.state()
    
# Print your results
print(apply_hxh(0))
print(apply_hxh(1))

[1.+0.j 0.+0.j]
[ 0.+0.j -1.+0.j]


# It's Just a Phase

**Z gate**

The Z gate is a quantum gate that applies a phase rotation to a qubit. Specifically, it multiplies the $|1\rangle$ state by $-1$, leaving the $|0\rangle$ state unchanged.
The Z gate can be represented by the following matrix:

$Z = \begin{bmatrix} 1 & 0 \\  0 & -1 \end{bmatrix}$

It is useful for creating quantum algorithms that rely on phase information.

**S gate (half-phase shift)**

The S gate is a quantum gate that applies a π/2 phase rotation to a qubit.
The S gate can be mathematically represented by the following matrix:

$S = \begin{bmatrix} 1 & 0 \\  0 & i \end{bmatrix}$

**T gate (quarter-phase shift)**

The T gate is a quantum gate that applies a π/4 phase rotation to a qubit.
The T gate can be represented as:

$T = \begin{bmatrix} 1 & 0 \\  0 & e^{i \pi \over 4} \end{bmatrix}$


In [17]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def apply_z_to_plus():
    # CREATE THE |+> STATE
    qml.Hadamard(wires=0)
    # APPLY PAULI Z
    qml.PauliZ(wires=0)
    # RETURN THE STATE
    return qml.state()

print(apply_z_to_plus())


[ 0.70710678+0.j -0.70710678+0.j]


In [18]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def fake_z():
    # CREATE THE |+> STATE
    qml.Hadamard(wires=0)
    # APPLY RZ
    qml.RZ(qml.numpy.pi, wires=0)
    # RETURN THE STATE
    return qml.state()

In [19]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def many_rotations():
    # IMPLEMENT THE CIRCUIT
    qml.Hadamard(wires=0)
    qml.S(wires=0)
    qml.adjoint(qml.T)(wires=0)
    qml.RZ(0.3, 0)
    qml.adjoint(qml.S)(wires=0)
    # RETURN THE STATE

    return qml.state()

# From a Different Angle

The rotation gates, such as RZ, RX and RY, are a family of quantum gates that apply a rotation to a qubit around one of
its axes. These gates are parameterized by an angle.

* **RZ**: Applies a rotation around the z-axis:
 $$RZ(\theta) = \begin{bmatrix}e^{-i \theta \over 2} & 0 \\ 0 & e^{i \theta \over 2}\end{bmatrix}$$
* **RY**: Applies a rotation around the y-axis:
 $$RY(\theta) = \begin{bmatrix}cos(\theta) & -sin(\theta) \\ sin(\theta) & cos(\theta)\end{bmatrix}$$
* **RX**: Applies a rotation around the x-axis:
 $$RX(\theta) = \begin{bmatrix}cos(\theta) & -i\cdot sin(\theta) \\ -i\cdot sin(\theta) & cos(\theta)\end{bmatrix}$$

The rotation gates are parameterized by an angle θ, which can be any real number. The gate applies a rotation of θ around the specified
axis. 

In [20]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def apply_rx_pi(state):
    # APPLY RX(pi) AND RETURN THE STATE
    qml.RX(qml.numpy.pi, 0)
    return qml.state()

print(apply_rx_pi(0))
print(apply_rx_pi(1))

[6.123234e-17+0.j 0.000000e+00-1.j]
[6.123234e-17+0.j 0.000000e+00-1.j]


In [21]:
dev = qml.device("default.qubit", wires=1)


@qml.qnode(dev)
def apply_rx(theta, state):
    # APPLY RX(theta) AND RETURN THE STATE
    qml.RX(theta, 0)
    return qml.state()

# Code for plotting
angles = np.linspace(0, 4 * np.pi, 200)
output_states = np.array([apply_rx(t, 0) for t in angles])

In [22]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def apply_ry(theta, state):
    if state == 1:
        qml.PauliX(wires=0)
    # APPLY RY(theta) AND RETURN THE STATE
    qml.RY(theta, wires=0)
    return qml.state()

# Code for plotting
angles = np.linspace(0, 4 * np.pi, 200)
output_states = np.array([apply_ry(t, 0) for t in angles])

# Universal Gate Sets

These sets represent a collection of quantum gates that can be used to approximate any unitary operation on a 
quantum computer to any desired level of accuracy. Some examples of universal gate sets for single qubit gates are:
* $\{RZ, RY\}$
* $\{H, T\}$


In [23]:
dev = qml.device("default.qubit", wires=1)
# ADJUST THE VALUES OF PHI, THETA, AND OMEGA
phi, theta, omega = qml.numpy.pi/2, qml.numpy.pi/2, qml.numpy.pi/2

@qml.qnode(dev)
def hadamard_with_rz_rx():
    qml.RZ(phi, wires=0)
    qml.RX(theta, wires=0)
    qml.RZ(omega, wires=0)
    return qml.state()

In [24]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def convert_to_rz_rx():
    # IMPLEMENT THE CIRCUIT IN THE PICTURE USING ONLY RZ AND RX
    qml.RZ(qml.numpy.pi/2, wires=0)
    qml.RX(qml.numpy.pi/2, wires=0)
    qml.RZ(qml.numpy.pi/4, wires=0)
    qml.RX(-qml.numpy.pi, wires=0)
    qml.RZ(qml.numpy.pi/2, wires=0)
    return qml.state()

In [25]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def unitary_with_h_and_t():
    # APPLY ONLY H AND T TO PRODUCE A CIRCUIT THAT EFFECTS THE GIVEN MATRIX
    qml.Hadamard(wires=0)
    qml.T(wires=0)
    qml.Hadamard(wires=0)
    qml.T(wires=0)
    qml.T(wires=0)
    qml.Hadamard(wires=0)
    return qml.state()

# Prepare Yourself

State preparation refers to the process of preparing a quantum system in a specific state. This can be thought of as initializing the quantum system to have a particular probability distribution over all possible states.

State preparation is an essential step in almost every quantum algorithm and protocol. It involves applying a sequence of quantum gates (such as `rotations`, `Hadamard` gates, or `CNOT` gates) to the quantum system to prepare it in the desired state.

In [26]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def prepare_state():
    # APPLY OPERATIONS TO PREPARE THE TARGET STATE
    qml.Hadamard(wires=0)
    qml.RZ(qml.numpy.pi*5/4,wires=0)

    return qml.state()

In [27]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def prepare_state():
    # APPLY OPERATIONS TO PREPARE THE TARGET STATE
    qml.Hadamard(wires=0)
    qml.RZ(np.pi/3,wires=0)
    qml.Hadamard(wires=0)
    return qml.state()


In [28]:
v = np.array([0.52889389 - 0.14956775j, 0.67262317 + 0.49545818j])
# CREATE A DEVICE
dev = qml.device("default.qubit", wires=1)

# CONSTRUCT A QNODE THAT USES qml.MottonenStatePreparation
# TO PREPARE A QUBIT IN STATE V, AND RETURN THE STATE

@qml.qnode(dev)
def prepare_state(state=v):
    qml.MottonenStatePreparation(state, wires=0)
    return qml.state()

# This will draw the quantum circuit and allow you to inspect the output gates
print(prepare_state(v))
print()
print(qml.draw(prepare_state, expansion_strategy="device")(v))

[0.52889389-0.14956775j 0.67262317+0.49545818j]

0: ──RY(1.98)──RZ(0.91)──GlobalPhase(-0.18)─┤  State


# Measurements

**Quantum Measurement**

In quantum mechanics, a measurement is the process of collapsing the wave function of a quantum system into one of its possible
eigenstates. This is typically achieved by interacting with the system in such a way that it causes the wave function to collapse.

When we measure a quantum state, we are essentially asking which basis (or set of orthogonal states) the system is currently in. For example,
if we have a qubit in a superposition of $|0\rangle$ and $|1\rangle$ states ($|+\rangle$ state), measuring it will cause it to collapse into either $|0\rangle$ or $|1\rangle$ with equal probability.

In [29]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def apply_h_and_measure(state):
    if state == 1:
        qml.PauliX(wires=0)
    # APPLY HADAMARD AND MEASURE
    qml.Hadamard(wires=0)
    return qml.probs(wires=0)

print(apply_h_and_measure(0))
print(apply_h_and_measure(1))

[0.5 0.5]
[0.5 0.5]


**Measuring in Different Basis**

When we measure a quantum state, we are typically measuring it in the standard basis ($|0\rangle$ and $|1\rangle$). However, there are situations where we might want to measure a qubit in a different basis. This can be achieved by applying a unitary transformation (such as a rotation gate) to the qubit before measurement.

In [30]:
# WRITE A QUANTUM FUNCTION THAT PREPARES (1/2)|0> + i(sqrt(3)/2)|1>
def prepare_psi():
    qml.Hadamard(wires=0)
    qml.RZ(-np.pi*2/3,wires=0)
    qml.Hadamard(wires=0)

# WRITE A QUANTUM FUNCTION THAT SENDS BOTH |0> TO |y_+> and |1> TO |y_->
def y_basis_rotation():
    qml.Hadamard(wires=0)
    qml.S(wires=0)

In [31]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def measure_in_y_basis():
    # PREPARE THE STATE
    prepare_psi()
    # PERFORM THE ROTATION BACK TO COMPUTATIONAL BASIS
    qml.adjoint(y_basis_rotation)()
    # RETURN THE MEASUREMENT OUTCOME PROBABILITIES
    return qml.probs(wires=0)

print(measure_in_y_basis())

[0.9330127 0.0669873]


# What Did You Expect?

**Observables**

In quantum mechanics, an observable is a physical quantity that can be measured in a quantum system. Examples of observables include energy
`E`, momentum `p`, position `x` and spin `S`. These quantities are represented by mathematical operators that act on the wave function of the system.

When we measure an observable in a quantum system, we are essentially asking which eigenvalue of the operator corresponds (has a greater probability) to the actual value of the observable.

In [32]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def circuit():
    # IMPLEMENT THE CIRCUIT IN THE PICTURE AND MEASURE PAULI Y
    qml.RX(qml.numpy.pi/4,wires=0)
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=0)
    return qml.expval(qml.PauliY(wires=0))

print(circuit())

-0.7071067811865471


In [33]:
# An array to store your results
shot_results = []

# Different numbers of shots
shot_values = [100, 1000, 10000, 100000, 1000000]

for shots in shot_values:
    # CREATE A DEVICE, CREATE A QNODE, AND RUN IT
    dev = qml.device("default.qubit", wires=1, shots=shots)
    
    @qml.qnode(dev)
    def circuit():
        # IMPLEMENT THE CIRCUIT IN THE PICTURE AND MEASURE PAULI Y
        qml.RX(qml.numpy.pi/4,wires=0)
        qml.Hadamard(wires=0)
        qml.PauliZ(wires=0)
        return qml.expval(qml.PauliY(wires=0))

    # STORE RESULT IN SHOT_RESULTS ARRAY
    shot_results.append(circuit())

print(qml.math.unwrap(shot_results))

[-0.7, -0.716, -0.7088, -0.70756, -0.707192]


In [34]:
dev = qml.device("default.qubit", wires=1, shots=100000)

@qml.qnode(dev)
def circuit():
    qml.RX(np.pi / 4, wires=0)
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=0)
    # RETURN THE MEASUREMENT SAMPLES OF THE CORRECT OBSERVABLE

    return qml.sample(qml.PauliY(wires=0))

def compute_expval_from_samples(samples):
    estimated_expval = 0
    # USE THE SAMPLES TO ESTIMATE THE EXPECTATION VALUE
    estimated_expval = np.sum(samples)/len(samples)
    return estimated_expval

samples = circuit()
print(compute_expval_from_samples(samples))

-0.70688


The `variance_scaling` function analyzes how the variance of an expectation value changes as the number of measurements (shots) increases. It compares actual experimental results to theoretically estimated values, providing insight into the effect of shot count on quantum circuit outcomes.

In [35]:
def variance_experiment(n_shots):
    # To obtain a variance, we run the circuit multiple times at each shot value.
    n_trials = 100

    # CREATE A DEVICE WITH GIVEN NUMBER OF SHOTS
    dev = qml.device("default.qubit", wires=1, shots=n_shots)
    # DECORATE THE CIRCUIT BELOW TO CREATE A QNODE
    @qml.qnode(dev)
    def circuit():
        qml.Hadamard(wires=0)
        return qml.expval(qml.PauliZ(wires=0))

    # RUN THE QNODE N_TRIALS TIMES AND RETURN THE VARIANCE OF THE RESULTS
    var = np.var([circuit() for i in range(n_trials)])
    return var


def variance_scaling(n_shots):
    estimated_variance = 0

    # ESTIMATE THE VARIANCE BASED ON SHOT NUMBER
    estimated_variance = 1/n_shots
    return estimated_variance


# Various numbers of shots; you can change this
shot_vals = [10, 20, 40, 100, 200, 400, 1000, 2000, 4000]

# Used to plot your results
results_experiment = [variance_experiment(shots) for shots in shot_vals]
results_scaling = [variance_scaling(shots) for shots in shot_vals]

# Multi-Qubit Systems

**Tensor Product**

In a multi-qubit system, each qubit can exist in one of two states: $|0\rangle$ or $|1\rangle$. When we have multiple qubits, we need to describe the state of all of them simultaneously.

The tensor product is a way of combining two or more vectors (or operators) into a new vector (or operator). For qubits, it's used to
combine their individual states into a single state that describes the entire system.

Mathematically, if we have two qubits A and B with states $|0\rangle_A$ and $|1\rangle_B$ respectively, their tensor product is written as:

$|\psi \rangle = |0\rangle_A \otimes |0\rangle_B$

This means that the state of qubit A ($|0\rangle_A$) and the state of qubit B ($|1\rangle_B$) are combined to form a new state $|\psi \rangle$, where each qubit's state is preserved.

**Multi-qubit Basis**

The multi-qubit basis is an extension of the single-qubit basis. For a single qubit, we have two states: $|0\rangle_A$ and $|1\rangle_A$. When we move to multiple qubits, we need a basis that can describe all possible combinations of states.

One two-qubit basis is the computational basis, which consists of:

$$|00\rangle$$
$$|01\rangle$$
$$|10\rangle$$
$$|11\rangle$$

Each state in this basis represents a specific combination of qubit states. For example, $|01\rangle$ means that qubit $A$ is in state $|0\rangle$ and qubit $B$ is in state $|1\rangle$.

In [36]:
num_wires = 3
dev = qml.device("default.qubit", wires=num_wires)

@qml.qnode(dev)
def make_basis_state(basis_id):
    # CREATE THE BASIS STATE
    pos = 0
    for i in np.binary_repr(basis_id,num_wires):
        if int(i):
            qml.PauliX(wires=pos)
        pos += 1
    return qml.state()

basis_id = 3
print(f"Output state = {make_basis_state(basis_id)}")

Output state = [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [37]:
# Creates a device with *two* qubits
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def two_qubit_circuit():
    # PREPARE |+>|1>
    qml.Hadamard(wires=0)
    qml.PauliX(wires=1)
    # RETURN TWO EXPECTATION VALUES, Y ON FIRST QUBIT, Z ON SECOND QUBIT
    
    return qml.expval(qml.PauliY(wires=0)), qml.expval(qml.PauliZ(wires=1))

print(two_qubit_circuit())

(tensor(0., requires_grad=True), tensor(-1., requires_grad=True))


In [38]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def create_one_minus():
    # PREPARE |1>|->
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    qml.Hadamard(wires=1)
    # RETURN A SINGLE EXPECTATION VALUE Z \otimes X
    return qml.expval(qml.PauliZ(wires=0) @ qml.PauliX(wires=1))

print(create_one_minus())

0.9999999999999996


**Separable Operations**

In the context of quantum computing, separable operations refer to any operation that can be written as a tensor product of individual operators acting on each qubit separately. In other words, if an operation O can be expressed as:

$O = O_A \otimes O_B$

where $O_A$ and $O_B$ are operators acting on qubits A and B, respectively, then the operation O is said to be separable.

In [39]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit_1(theta):
    qml.RX(theta, 0)
    qml.RY(2*theta, 1)
    
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

@qml.qnode(dev)
def circuit_2(theta):
    qml.RX(theta, 0)
    qml.RY(2*theta, 1)
    
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))


def zi_iz_combination(ZI_results, IZ_results):
    combined_results = np.zeros(len(ZI_results))

    combined_results = np.multiply(ZI_results, IZ_results)
    return combined_results

theta = np.linspace(0, 2 * np.pi, 100)

# Run circuit 1, and process the results
circuit_1_results = np.array([circuit_1(t) for t in theta])

ZI_results = circuit_1_results[:, 0]
IZ_results = circuit_1_results[:, 1]
combined_results = zi_iz_combination(ZI_results, IZ_results)

# Run circuit 2
ZZ_results = np.array([circuit_2(t) for t in theta])

# All Tied Up

**Entangled States**

In quantum mechanics, entangled states are a fundamental concept that goes beyond the classical understanding of probability and correlation. 

A classic example of entangled states is the Bell state:

$$| \psi \rangle = {{(|00 \rangle + |11 \rangle)} \over \sqrt{2}}$$

In this state, the qubits A and B are entangled such that measuring one qubit's state instantly determines the state of the other. If you measure qubit `A` to be $0$, then qubit `B` must be $0$ (and vice versa).

**CNOT Gate**

The CNOT gate is a controlled-NOT operation that flips the target qubit if the control qubit is in state 1. Here are the matrices for each possible combination:

| Control Qubit | Target Qubit | Resulting State |
| :--- | :---: | ---: |
| $|0\rangle$ |$|0\rangle$  |$|00\rangle$
| $|0\rangle$ |$|1\rangle$  |$|01\rangle$
| $|1\rangle$ |$|0\rangle$  |$|11\rangle$
| $|1\rangle$ |$|1\rangle$  |$|10\rangle$


The matrix representation of the CNOT gate is:

$$CNOT = \begin{bmatrix} 1&0&0&0 \\ 0&1&0&0 \\ 0&0&0&1 \\ 0&0&1&0 \end{bmatrix}$$

**CZ Gate**

The CZ gate, also known as the controlled-Z operation, applies a phase flip to the target qubit if the control qubit is in state 1. Here are the matrices for each possible combination:

| Control Qubit | Target Qubit | Resulting State |
| :--- | :---: | ---: |
| $|0\rangle$ |$|0\rangle$  |$|00\rangle$
| $|0\rangle$ |$|1\rangle$  |$|01\rangle$
| $|1\rangle$ |$|0\rangle$  |$|10\rangle$
| $|1\rangle$ |$|1\rangle$  |$-|11\rangle$

The matrix representation of the CZ gate is:

$$CZ = \begin{bmatrix} 1&0&0&0 \\ 0&1&0&0 \\ 0&0&1&0 \\ 0&0&0&-1 \end{bmatrix}$$

In [40]:
num_wires = 2
dev = qml.device("default.qubit", wires=num_wires)

@qml.qnode(dev)
def apply_cnot(basis_id):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=num_wires)]
    qml.BasisStatePreparation(bits, wires=[0, 1])

    # APPLY THE CNOT
    qml.CNOT(wires=[0, 1])
    return qml.state()

# REPLACE THE BIT STRINGS VALUES BELOW WITH THE CORRECT ONES
cnot_truth_table = {"00": "00", "01": "01", "10": "11", "11": "10"}

# Run your QNode with various inputs to help fill in your truth table
print(apply_cnot(0))

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]


In [41]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def apply_h_cnot():
    # APPLY THE OPERATIONS IN THE CIRCUIT
    qml.Hadamard(0)
    qml.CNOT([0,1])
    return qml.state()


print(apply_h_cnot())
# SET THIS AS 'separable' OR 'entangled' BASED ON YOUR OUTCOME
state_status = "entangled"

[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [42]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def controlled_rotations(theta, phi, omega):
    # APPLY THE OPERATIONS IN THE CIRCUIT AND RETURN MEASUREMENT PROBABILITIES
    qml.Hadamard(0)
    qml.CRX(theta, [0,1])
    qml.CRY(phi, [1,2])
    qml.CRZ(omega,[2,0])
    return qml.probs()

theta, phi, omega = 0.1, 0.2, 0.3
print(controlled_rotations(theta, phi, omega))

[5.00000000e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.98751041e-01 0.00000000e+00 1.23651067e-03 1.24480103e-05]


# We've Got It Under Control

**SWAP Gate**

The SWAP gate is a quantum gate that swaps the states of two qubits. In other words, it exchanges the state of one qubit with the state of another.

The matrix representation of the SWAP gate is:

$$SWAP = \begin{bmatrix} 1&0&0&0 \\ 0&0&1&0 \\ 0&1&0&0 \\ 0&0&0&1 \end{bmatrix}$$

**Toffoli Gate**

The Toffoli gate, also known as the controlled-controlled-NOT (CCNOT) gate, is a quantum gate that flips the state of one qubit if both control qubits are in the state 1.

If we consider three qubits `A`, `B` and `C`, the Toffoli gate would transform the states $|000\rangle$, $|010\rangle$, $|100\rangle$ and $|110\rangle$ as follows:

|Control State | Target State | Resulting State |
| :--- | :---: | ---: |
| $|00\rangle$ | $|0\rangle$ |$|000\rangle$
| $|01\rangle$ | $|0\rangle$ |$|010\rangle$
| $|10\rangle$ | $|0\rangle$ |$|100\rangle$
| $|11\rangle$ | $|0\rangle$ |$|111\rangle$

The matrix representation of the Toffoli gate is:

$$Toffoli = \begin{bmatrix} 1&0&0&0&0&0&0&0 \\ 0&1&0&0&0&0&0&0 \\ 0&0&1&0&0&0&0&0 \\ 0&0&0&1&0&0&0&0 \\
                            0&0&0&0&1&0&0&0 \\ 0&0&0&0&0&1&0&0 \\ 0&0&0&0&0&0&0&1 \\ 0&0&0&0&0&0&1&0 \end{bmatrix}$$



In [43]:
dev = qml.device("default.qubit", wires=2)

# Prepare a two-qubit state; change up the angles if you like
#phi, theta, omega = 1.2, 2.3, 3.4
states = np.sqrt([0.1,0.2,0.3,0.4])

@qml.qnode(device=dev)
def true_cz(states):
    #prepare_states(phi, theta, omega)
    qml.StatePrep(states, wires=range(2))
    # IMPLEMENT THE REGULAR CZ GATE HERE
    qml.CZ([0, 1])
    return qml.state()

@qml.qnode(dev)
def imposter_cz(states):
    #prepare_states(phi, theta, omega)
    qml.StatePrep(states, wires=range(2))
    # IMPLEMENT CZ USING ONLY H AND CNOT
    qml.Hadamard(1)
    qml.CNOT([0, 1])
    qml.Hadamard(1)
    return qml.state()


print(f"True CZ output state {true_cz(states)}")
print(f"Imposter CZ output state {imposter_cz(states)}")

True CZ output state [ 0.31622777+0.j  0.4472136 +0.j  0.54772256+0.j -0.63245553+0.j]
Imposter CZ output state [ 0.31622777+0.j  0.4472136 +0.j  0.54772256+0.j -0.63245553+0.j]


In [44]:
dev = qml.device("default.qubit", wires=2)

# Prepare a two-qubit state; change up the angles if you like
#phi, theta, omega = 1.2, 2.3, 3.4
states = np.sqrt([0.1,0.2,0.3,0.4])

@qml.qnode(dev)
def apply_swap(states):
    #prepare_states(phi, theta, omega)
    qml.StatePrep(states, wires=range(2))
    # IMPLEMENT THE REGULAR SWAP GATE HERE
    qml.SWAP([0, 1])
    return qml.state()

@qml.qnode(dev)
def apply_swap_with_cnots(states):
    #prepare_states(phi, theta, omega)
    qml.StatePrep(states, wires=range(2))
    # IMPLEMENT THE SWAP GATE USING A SEQUENCE OF CNOTS
    qml.CNOT([0, 1])
    qml.CNOT([1, 0])
    qml.CNOT([0, 1])
    return qml.state()

print(f"Regular SWAP state = {apply_swap(states)}")
print(f"CNOT SWAP state = {apply_swap_with_cnots(states)}")

Regular SWAP state = [0.31622777+0.j 0.54772256+0.j 0.4472136 +0.j 0.63245553+0.j]
CNOT SWAP state = [0.31622777+0.j 0.54772256+0.j 0.4472136 +0.j 0.63245553+0.j]


In [45]:
dev = qml.device("default.qubit", wires=3)

# Prepare first qubit in |1>, and arbitrary states on the second two qubits
#phi, theta, omega = 1.2, 2.3, 3.4
states = np.array([0.5, 0.5, 0.5, 0.5])

# A helper function just so you can visualize the initial state
# before the controlled SWAP occurs.
@qml.qnode(dev)
def no_swap(states):
    #prepare_states(phi, theta, omega)
    qml.StatePrep(states, wires=range(2))
    return qml.state()

@qml.qnode(dev)
def controlled_swap(states):
    #prepare_states(phi, theta, omega)
    qml.StatePrep(states, wires=range(2))
    # PERFORM A CONTROLLED SWAP USING A SEQUENCE OF TOFFOLIS
    qml.Toffoli([0, 1, 2])
    qml.Toffoli([0, 2, 1])
    qml.Toffoli([0, 1, 2])
    return qml.state()

print(no_swap(states))
print(controlled_swap(states))

[0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j]
[0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0.5+0.j 0. +0.j 0. +0.j]


In [46]:
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def four_qubit_mcx():
    # IMPLEMENT THE CIRCUIT ABOVE USING A 4-QUBIT MULTI-CONTROLLED X
    qml.Hadamard(0)
    qml.Hadamard(1)
    qml.Hadamard(2)
    qml.MultiControlledX(wires=[0, 1, 2, 3])
    return qml.state()

print(four_qubit_mcx())

[0.35355339+0.j 0.        +0.j 0.35355339+0.j 0.        +0.j
 0.35355339+0.j 0.        +0.j 0.35355339+0.j 0.        +0.j
 0.35355339+0.j 0.        +0.j 0.35355339+0.j 0.        +0.j
 0.35355339+0.j 0.        +0.j 0.        +0.j 0.35355339+0.j]


In [47]:
# Wires 0, 1, 2 are the control qubits
# Wire 3 is the auxiliary qubit
# Wire 4 is the target
dev = qml.device("default.qubit", wires=5)

@qml.qnode(dev)
def four_qubit_mcx_only_tofs():
    # We will initialize the control qubits in state |1> so you can see
    # how the output state gets changed.
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    qml.PauliX(wires=2)

    # IMPLEMENT A 3-CONTROLLED NOT WITH TOFFOLIS
    qml.Toffoli([0, 1, 3])
    qml.Toffoli([2, 3, 4])
    qml.Toffoli([0, 1, 3])
    return qml.state()

# print(four_qubit_mcx_only_tofs())

# Multi-Qubit Gate Challenge

In [48]:
dev = qml.device("default.qubit", wires=2)

# Starting from the state |00>, implement a PennyLane circuit
# to construct each of the Bell basis states.

@qml.qnode(dev)
def prepare_psi_plus():
    # PREPARE (1/sqrt(2)) (|00> + |11>)
    qml.Hadamard(0)
    qml.CNOT([0, 1])
    return qml.state()


@qml.qnode(dev)
def prepare_psi_minus():
    # PREPARE (1/sqrt(2)) (|00> - |11>)
    qml.Hadamard(0)
    qml.CNOT([0, 1])
    qml.PauliZ(1)
    return qml.state()


@qml.qnode(dev)
def prepare_phi_plus():
    qml.Hadamard(0)
    qml.CNOT([0, 1])
    qml.PauliX(1)
    # PREPARE  (1/sqrt(2)) (|01> + |10>)

    return qml.state()

@qml.qnode(dev)
def prepare_phi_minus():
    qml.Hadamard(0)
    qml.CNOT([0, 1])
    qml.PauliZ(1)
    qml.PauliX(1)
    # PREPARE  (1/sqrt(2)) (|01> - |10>)

    return qml.state()

psi_plus = prepare_psi_plus()
psi_minus = prepare_psi_minus()
phi_plus = prepare_phi_plus()
phi_minus = prepare_phi_minus()

# Uncomment to print results
print(f"|ψ_+> = {psi_plus}")
print(f"|ψ_-> = {psi_minus}")
print(f"|ϕ_+> = {phi_plus}")
print(f"|ϕ_-> = {phi_minus}")

|ψ_+> = [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
|ψ_-> = [ 0.70710678+0.j  0.        +0.j  0.        +0.j -0.70710678+0.j]
|ϕ_+> = [0.        +0.j 0.70710678+0.j 0.70710678+0.j 0.        +0.j]
|ϕ_-> = [ 0.        +0.j  0.70710678+0.j -0.70710678+0.j  0.        +0.j]


In [49]:
dev = qml.device("default.qubit", wires=3)

# State of first 2 qubits
state = [0, 1]

@qml.qnode(device=dev)
def apply_control_sequence(state):
    # Set up initial state of the first two qubits
    if state[0] == 1:
        qml.PauliX(wires=0)
    if state[1] == 1:
        qml.PauliX(wires=1)

    # Set up initial state of the third qubit - use |->
    # so we can see the effect on the output
    qml.PauliX(wires=2)
    qml.Hadamard(wires=2)

    # IMPLEMENT THE MULTIPLEXER
    # IF STATE OF FIRST TWO QUBITS IS 01, APPLY X TO THIRD QUBIT
    qml.PauliX(0)
    qml.Toffoli([0, 1, 2])
    qml.PauliX(0)
    # IF STATE OF FIRST TWO QUBITS IS 10, APPLY Z TO THIRD QUBIT
    qml.PauliX(1)
    qml.Hadamard(2)
    qml.Toffoli([0, 1, 2])
    qml.Hadamard(2)
    qml.PauliX(1)
    # IF STATE OF FIRST TWO QUBITS IS 11, APPLY Y TO THIRD QUBIT
    qml.adjoint(qml.S)(2)
    qml.Toffoli([0, 1, 2])
    qml.S(2)
    return qml.state()

print(apply_control_sequence(state))

[ 0.        +0.j  0.        +0.j -0.70710678+0.j  0.70710678+0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
